###Reference
1. The codes for DeepSORT was adapted from the GitHub repo: https://github.com/nwojke/deep_sort

2. The codes for Skeleton FittingTF-PoseEstimator was adapted from the GitHub repo: https://github.com/ildoonet/tf-pose-estimation

3. The codes for ST-DenseNet was adapted from the GitHub repo: https://github.com/GalDude33/DenseNetFCN-3D

4. The codes for YOLOv3 was adapted from the GitHub repo: https://github.com/zzh8829/yolov3-tf2

## 1. Run this to install dependencies

In [ ]:
%cd Volvo-DataX/tf-pose-estimation
! pip3 install -r requirements.txt
%cd tf_pose/pafprocess
! sudo apt install swig
!swig -python -c++ pafprocess.i && python3 setup.py build_ext --inplace

/content/Volvo-DataX/tf-pose-estimation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ppwwyyxx/tensorpack.git to /tmp/pip-req-build-g31p48rf
  Running command git clone --filter=blob:none --quiet https://github.com/ppwwyyxx/tensorpack.git /tmp/pip-req-build-g31p48rf
  Resolved https://github.com/ppwwyyxx/tensorpack.git to commit a9a2660d9ebe8fe4f32693b59f1e003687716d81
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=d21d49fa65c01319cd7d5ee3b205d4f15a06c128f4ca9f5d62b63ac66c0205ac
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
  Created wheel for tensorpack: filename=ten

/content/Volvo-DataX/tf-pose-estimation/tf_pose/pafprocess
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 24 not upgraded.
Need to get 1,086 kB of archives.
After this operation, 5,413 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 swig4.0 amd64 4.0.1-5build1 [1,081 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 swig all 4.0.1-5build1 [5,528 B]
Fetched 1,086 kB in 1s (1,302 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###5. Code

In [ ]:
# run this
%cd /content/Volvo-DataX

try:
  %tensorflow_version 2.x
except Exception:
  pass
import glob

import sys #Run this
from absl import app, logging, flags
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

tf.compat.v1.disable_eager_execution()
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

flags.DEFINE_string('classes', 'data/coco.names', 'path to classes file')
flags.DEFINE_string('weights', '/content/drive/My Drive/datax_volvo_additional_files/yolov3_train_5.tf','path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_integer('num_classes', 1, 'number of classes in the model')
flags.DEFINE_string('video', 'data/JAAD_test_video_0339.mp4','path to video file or number for webcam)')
flags.DEFINE_string('output','Result_model_D.mp4', 'path to output video')
flags.DEFINE_string('output_format', 'mp4v', 'codec used in VideoWriter when saving video to file')

app._run_init(['yolov3'], app.parse_flags_with_usage)


/content/Volvo-DataX
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


['yolov3']

In [ ]:
%cd /content/Volvo-DataX/deep_sort
from ds_tools.generate_detections import create_box_encoder
from ds_application_util import preprocessing
from ds_deep_sort import nn_matching
from ds_deep_sort.detection import Detection
from ds_deep_sort.tracker import Tracker

%cd /content/Volvo-DataX/tf-pose-estimation
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
from tf_pose.estimator import Human
model = TfPoseEstimator(get_graph_path('egen_jaad_1_5'), target_size=(100, 100))

%cd /content/Volvo-DataX

nms_max_overlap = 1.0
max_cosine_distance = 0.2
nn_budget = None
encoder = create_box_encoder('mars-small128.pb', batch_size=32)
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = Tracker(metric)

with open('densenet_model.json', 'r') as json_file:
    json_savedModel= json_file.read()

model_j = tf.keras.models.model_from_json(json_savedModel)
model_j.load_weights('densenet_2.hdf5')

def pred_func(X_test):
  predictions = model_j.predict(X_test[0:1], verbose=0)
  Y = np.argmax(predictions[0], axis=0)

  return Y


W0504 06:51:47.193216 140519420950336 deprecation.py:364] From /usr/local/lib/python3.10/dist-packages/tensorflow/python/compat/v2_compat.py:107: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


/content/Volvo-DataX/deep_sort
/content/Volvo-DataX/tf-pose-estimation


/content/utils/linear_assignment_.py:18: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(
[2023-05-04 06:51:47,796] [TfPoseEstimator] [INFO] loading graph from /content/drive/My Drive/datax_volvo_additional_files/graph_opt.pb(default size=100x100)
I0504 06:51:47.796014 140519420950336 estimator.py:320] loading graph from /content/drive/My Drive/datax_volvo_additional_files/graph_opt.pb(default size=100x100)


TfPoseEstimator/preprocess_divide/y
TfPoseEstimator/preprocess_subtract/y
TfPoseEstimator/concat_stage2/axis
TfPoseEstimator/concat_stage3/axis
TfPoseEstimator/concat_stage4/axis
TfPoseEstimator/concat_stage5/axis
TfPoseEstimator/concat_stage6/axis
TfPoseEstimator/Openpose/concat_stage7/axis
TfPoseEstimator/Mconv1_stage2_L1/biases/read/_0__cf__0
TfPoseEstimator/Mconv1_stage2_L1/weights/read/_1__cf__1
TfPoseEstimator/Mconv1_stage2_L2/biases/read/_2__cf__2
TfPoseEstimator/Mconv1_stage2_L2/weights/read/_3__cf__3
TfPoseEstimator/Mconv1_stage3_L1/biases/read/_4__cf__4
TfPoseEstimator/Mconv1_stage3_L1/weights/read/_5__cf__5
TfPoseEstimator/Mconv1_stage3_L2/biases/read/_6__cf__6
TfPoseEstimator/Mconv1_stage3_L2/weights/read/_7__cf__7
TfPoseEstimator/Mconv1_stage4_L1/biases/read/_8__cf__8
TfPoseEstimator/Mconv1_stage4_L1/weights/read/_9__cf__9
TfPoseEstimator/Mconv1_stage4_L2/biases/read/_10__cf__10
TfPoseEstimator/Mconv1_stage4_L2/weights/read/_11__cf__11
TfPoseEstimator/Mconv1_stage5_L1/bias

W0504 06:52:06.396773 140519420950336 deprecation.py:641] From /usr/local/lib/python3.10/dist-packages/tensorflow/python/util/deprecation.py:576: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with distribution=normal is deprecated and will be removed in a future version.
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


/content/Volvo-DataX


W0504 06:52:06.564311 140519420950336 deprecation.py:364] From /usr/local/lib/python3.10/dist-packages/keras/layers/normalization/batch_normalization.py:581: _colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [ ]:


FLAGS.yolo_iou_threshold = 0.5
FLAGS.yolo_score_threshold = 0.5

color = (255, 0, 0)
thickness = 2

yolo = YoloV3(classes=FLAGS.num_classes)

yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded')

class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')

resize_out_ratio = 4.0

def run_model():

  print('Processing started.......')

  fps_time = 0

  try:
      vid = cv2.VideoCapture(int(FLAGS.video))
  except:
      vid = cv2.VideoCapture(FLAGS.video)

  out = None

  if FLAGS.output:
      # by default VideoCapture returns float instead of int
      width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
      height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
      fps = int(vid.get(cv2.CAP_PROP_FPS))
      codec = cv2.VideoWriter_fourcc(*FLAGS.output_format)
      out = cv2.VideoWriter(FLAGS.output, codec, fps, (width, height))

  frame = 0
  rolling_data={}
  fps_time=0

  while True:

    _, img = vid.read() # reading the image

    if img is None:
        break
        logging.warning("Empty Frame")
        time.sleep(0.1)
        continue
    frame += 1
    currFrame = int(vid.get(cv2.CAP_PROP_POS_FRAMES))
    fps = vid.get(cv2.CAP_PROP_FPS)


    img_in = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_orig = np.copy(img)
    img_in = tf.expand_dims(img_in, 0)
    img_in = transform_images(img_in, FLAGS.size)

    t1 = time.time()
    boxes, scores, classes, nums = yolo.predict(img_in, steps=1) # yolo
    t2 = time.time()

    # for 80 cls yolo

    boxes = boxes[:,:nums[0],:].reshape(nums[0], 4)[classes[0][:nums[0]] == 0]
    scores = scores[0][:nums[0]][classes[0][:nums[0]] == 0]
    nums = len(boxes)

    wh = np.flip(img.shape[0:2])
    bbtlwh = []
    for i in range(nums):


      x1y1 = tuple((np.array(boxes[i][0:2]) * wh).astype(np.int32))
      x1 = x1y1[0]
      y1 = x1y1[1]
      x2y2 = tuple((np.array(boxes[i][2:4]) * wh).astype(np.int32))
      bbwh = (x2y2[0]-x1y1[0], x2y2[1]-x1y1[1])
      w = bbwh[0]
      h = bbwh[1]
      bbtlwh.append([x1,y1,w,h])

    features = encoder(img, bbtlwh) # deepsort input
    detections = [Detection(box, conf, feat) for box, conf, feat in zip(bbtlwh, scores, features)] #deep sort output

    # Update tracker.
    tracker.predict()
    tracker.update(detections)

    tracked_bbox = []
    ids = []

    for track in tracker.tracks:

      if not track.is_confirmed() or track.time_since_update > 1:
        continue
      tracked_bbox.append(track.to_tlwh())
      ids.append(track.track_id)


    for i in range(len(tracked_bbox)): # densenet

      # Show tracker output
      x, y, w, h = tracked_bbox[i]
      x = int(x)
      y = int(y)
      w = int(w)
      h = int(h)

      # plot the skeletons
      try:
        cropped = img_orig[y:y + h, x:x + w]
        humans = model.inference(cropped, resize_to_default=(w > 0 and h > 0), upsample_size=resize_out_ratio)
        humans.sort(key=lambda human: human.score, reverse=True)
        skelett = TfPoseEstimator.draw_humans(cropped, humans, imgcopy=True)
        img_orig[y:y + h, x:x + w] = skelett
        img_orig2 = img_orig

      except:
        img_orig2 = img_orig

      # looking for previous 16 frames data for a given pedestrian:

      intent = 0 #(default, the pedestrian is not crossing)


      if int(ids[i]) in list(rolling_data.keys()):

        if len(rolling_data[int(ids[i])]) == 16:

          seq = np.stack(np.array(rolling_data[int(ids[i])]),axis=2)
          seq = np.expand_dims(seq, axis=0)
          intent = pred_func(seq) # classification output

        else:

          seq = np.stack(np.array([rolling_data[int(ids[i])][-1]] * 16),axis=2)
          seq = np.expand_dims(seq, axis=0)
          intent = pred_func(seq) # classification output


      # risky pedestrian identification thru box color

      if intent == 1:
        color = (0, 0, 255) # Red -> Crossing

      else:
        color = (0, 255, 0) # green -> Not crossing

      fps_time = time.time()
      img = cv2.rectangle(img_orig2, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
      img = cv2.putText(img, 'TrackID ' + str(ids[i]), (x, y - 5), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 0, 0), thickness=2)
      img = cv2.putText(img,"Frame No: %d" % (frame),(10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255), 2)

      # storing the data for last 16 frames
# this is a test comment for running the command to shoot a video and thus this much typing
# now lest move to the second line which is again a timepass to shoot a video
#now the third line which is again a timepass to work and have fun while shooting no other reson to type this much
#why type when you can speak and why speak when you type
      try:

        if int(ids[i]) in list(rolling_data.keys()): # ID exists in dict

          if len(rolling_data[int(ids[i])]) < 16: # bboxes values for 16 frames

            cropped_seq = []
            cropped_img = cv2.resize(img_orig[y:h+y, x:w+x],(100,100))
            rolling_data[int(ids[i])].append(np.asarray(cropped_img)) # append the image

          else:

            del rolling_data[int(ids[i])][0] # delete oldest frame bbox and append latest frame bbox
            cropped_seq = []
            cropped_img = cv2.resize(img_orig[y:h+y, x:w+x],(100,100))
            rolling_data[int(ids[i])].append(np.asarray(cropped_img))

        else:

          cropped_seq = []
          cropped_img = cv2.resize(img_orig[y:h+y, x:w+x],(100,100))
          rolling_data[int(ids[i])] = [np.asarray(cropped_img)]

      except:
        pass


    if FLAGS.output:
      out.write(img)

    if cv2.waitKey(1) == ord('q'):
      break

  cv2.destroyAllWindows()
  print('\nProcessing completed.......!!!')
  print('Check video file in Volvo-DataX folder!')

  return

I0504 06:52:38.480125 140519420950336 <ipython-input-3-fdd16c3f0b95>:11] weights loaded
I0504 06:52:38.483900 140519420950336 <ipython-input-3-fdd16c3f0b95>:14] classes loaded



###6. Run this to obtain the output as a video file


In [ ]:
run_model()

Processing started.......


/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/content/utils/linear_assignment_.py:124: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(


To download the result video generated, search for it inside the Volvo-DataX folder in the left pane of the colab notebook and download the file from there.